In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})


import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw

def side_by_side_display(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    styles = [
        dict(selector="caption", props=[("caption-side", "center"), ("font-size", "100%"), ("color", )])]
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline; font-size:85%' ").set_precision(2).set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0"

    display(HTML(output))

    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, PoissonRegressor, Ridge, Lasso, RANSACRegressor, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, export_text, DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR, SVC, LinearSVC
import xgboost as xgb 


from joblib import dump, load
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split


import textwrap
def split_label(list_label):
    list_label = list(list_label)
    list_label = ["<br>".join(textwrap.wrap(t, width=12)) for t in list_label ]
    return list_label

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
df_pv_ = pd.read_csv('./feature/pv_28Feb22.csv')
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_MEMBERS','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [3]:
df_pv.head()

,TRO_MEMBERS,PV 2021-11,PV 2021-12,PV 2022-01,PV 2022-02
0,9990010000001200,46.0,38.0,13.0,19.0
1,9990010000001771,0.0,0.0,17.0,13.0
2,9990010000004582,0.0,0.0,2.0,0.0
3,9990010000008534,0.0,0.0,1.0,1.0
4,9990010000028444,0.0,0.0,2.0,26.0


In [4]:
df_re_g_u = pd.read_csv('./feature/rating_2022-03-01.csv')

In [5]:
df_re_g_u.head()

,TRO_MEMBERS,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_BELOW 2022-01,RATING_BELOW 2022-02,RATING_AVG 2021-11,RATING_AVG 2021-12,RATING_AVG 2022-01,RATING_AVG 2022-02
0,9990010000031123,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,9990010000169288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,9990010000180300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,9990010000191790,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,9990010000198083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [6]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2021-11-01', '2022-03-01', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop_test/{}_{}.csv'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_csv(pf, sep='\t', dtype='object'))
    
df_tto = pd.concat(df_tto)

# df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)

01Nov21 01Dec21
01Dec21 01Jan22
01Jan22 01Feb22
01Feb22 01Mar22


In [7]:
## USE THIS FOR GET PV DATA

df_mem_push = df_tto[['TRO_MEMBERS']].drop_duplicates()
from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

engine = create_engine(engine_stmt)

df = df_mem_push.astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()





In [8]:
df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')
df_tto['TRO_DH_SPESIAL'] = df_tto['TRO_DH_SPESIAL']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')


df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2021-11-01') \
            & (df_tto['TRO_DATE'] <= '2022-02-28')]

In [9]:
df_sla = pd.read_csv('./feature/sla_28Feb22.csv')
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [10]:
q = '''
SELECT 
	tc.TRO_MEMBERS, 
	(TO_DATE('2022/02/08', 'yyyy/mm/dd')) - zt.tgl_registrasi AS SINCE_REGIS
FROM 
	TEMP_CHURN tc
LEFT JOIN (
	select 
		char1 no_member, char4 jenis_kelamin, char5 status_nikah, date1 tgl_lahir, date2 tgl_registrasi
	from 
		report.ztampungan
	where 
		pk='MEMBERALFAGIFT2'
) zt ON zt.no_member = tc.TRO_MEMBERS 

'''

con = ds_db.connect_alfabi()

df_regis = pd.read_sql_query(q, con)

con.close()

You are connected


In [11]:
df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')

In [12]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_MEMBERS','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'first'
                              ''
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_MEMBERS','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

## unstack all feature in oshop

In [13]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

## merge with rating

In [14]:
df_re_g_u['TRO_MEMBERS'] = df_re_g_u['TRO_MEMBERS'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_MEMBERS', how='left').fillna(0)

## merge with product view

In [15]:
df_pv['TRO_MEMBERS'] = df_pv['TRO_MEMBERS'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_MEMBERS', how='left').fillna(0)

## merge with regis

In [16]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_MEMBERS')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [17]:
df_tto_re_pv.head()

,TRO_MEMBERS,TRO_NET 2021-11,TRO_NET 2021-12,TRO_NET 2022-01,TRO_NET 2022-02,TRO_DH_SPESIAL 2021-11,TRO_DH_SPESIAL 2021-12,TRO_DH_SPESIAL 2022-01,TRO_DH_SPESIAL 2022-02,TRO_NO_ORDER 2021-11,TRO_NO_ORDER 2021-12,TRO_NO_ORDER 2022-01,TRO_NO_ORDER 2022-02,SLA 2021-11,SLA 2021-12,SLA 2022-01,SLA 2022-02,SLA_LATE 2021-11,SLA_LATE 2021-12,SLA_LATE 2022-01,SLA_LATE 2022-02,TRO_VOUCHER_USAGE 2021-11,TRO_VOUCHER_USAGE 2021-12,TRO_VOUCHER_USAGE 2022-01,TRO_VOUCHER_USAGE 2022-02,TRO_PAYMENT_METHOD 2021-11,TRO_PAYMENT_METHOD 2021-12,TRO_PAYMENT_METHOD 2022-01,TRO_PAYMENT_METHOD 2022-02,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_BELOW 2022-01,RATING_BELOW 2022-02,RATING_AVG 2021-11,RATING_AVG 2021-12,RATING_AVG 2022-01,RATING_AVG 2022-02,PV 2021-11,PV 2021-12,PV 2022-01,PV 2022-02,SINCE_REGIS
2,9990010000001200,3117811.0,3217556.0,3436542.0,2403629.0,131400.0,138200.0,260000.0,82800.0,49.0,40.0,45.0,32.0,630.1,710.1,570.4,286.3,0.0,1.0,0.0,0.0,200000.0,85000.0,583000.0,254000.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,38.0,13.0,19.0,994.0
3,9990010000001771,0.0,0.0,154267.0,0.0,0.0,0.0,31500.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,14.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,929.0
4,9990010000004582,0.0,0.0,81454.0,0.0,0.0,0.0,30000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2062.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,806.0
5,9990010000008534,165315.0,776032.0,188468.0,157907.0,2200.0,185500.0,27400.0,34400.0,3.0,8.0,4.0,2.0,93.3,159.0,64.4,37.5,1.0,0.0,0.0,0.0,0.0,70000.0,0.0,12000.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,989.0
6,9990010000010072,0.0,0.0,0.0,75635.0,0.0,0.0,0.0,25400.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311.0


In [18]:
# df_tto_re_pv['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# df_tto_re_pv['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-10'] - df_tto_re_pv['TRO_NET 2021-09']
# df_tto_re_pv['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-09']

# df_tto_re_pv['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-10'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']
# df_tto_re_pv['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['PV 11-10'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# df_tto_re_pv['PV 10-09'] = df_tto_re_pv['PV 2021-10'] - df_tto_re_pv['PV 2021-09']
# df_tto_re_pv['PV 11-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-09']

# df_tto_re_pv['SLA 11-10'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# df_tto_re_pv['SLA 10-09'] = df_tto_re_pv['SLA 2021-10'] - df_tto_re_pv['SLA 2021-09']
# df_tto_re_pv['SLA 11-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-09']

# df_tto_re_pv['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# df_tto_re_pv['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-09']/df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [19]:
# print(df_ins[df_ins['label'] == 1]['NET_11-10'].median())
# print(df_ins[df_ins['label'] == 1]['NET_10-09'].median())
# print(df_ins[df_ins['label'] == 1]['NET_11-09'].median())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-10'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_10-09'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-09'].mean())

In [25]:
df_tto_re_pv[df_tto_re_pv['label'] == 1].head(20)

,TRO_MEMBERS,TRO_NET 2021-11,TRO_NET 2021-12,TRO_NET 2022-01,TRO_NET 2022-02,TRO_DH_SPESIAL 2021-11,TRO_DH_SPESIAL 2021-12,TRO_DH_SPESIAL 2022-01,TRO_DH_SPESIAL 2022-02,TRO_NO_ORDER 2021-11,TRO_NO_ORDER 2021-12,TRO_NO_ORDER 2022-01,TRO_NO_ORDER 2022-02,SLA 2021-11,SLA 2021-12,SLA 2022-01,SLA 2022-02,SLA_LATE 2021-11,SLA_LATE 2021-12,SLA_LATE 2022-01,SLA_LATE 2022-02,TRO_VOUCHER_USAGE 2021-11,TRO_VOUCHER_USAGE 2021-12,TRO_VOUCHER_USAGE 2022-01,TRO_VOUCHER_USAGE 2022-02,TRO_PAYMENT_METHOD 2021-11,TRO_PAYMENT_METHOD 2021-12,TRO_PAYMENT_METHOD 2022-01,TRO_PAYMENT_METHOD 2022-02,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_BELOW 2022-01,RATING_BELOW 2022-02,RATING_AVG 2021-11,RATING_AVG 2021-12,RATING_AVG 2022-01,RATING_AVG 2022-02,PV 2021-11,PV 2021-12,PV 2022-01,PV 2022-02,SINCE_REGIS,label
65,9990010000200894,551987.0,308488.0,74999.0,0.0,129700.0,74100.0,26700.0,0.0,2.0,3.0,1.0,0.0,49.8,64.9,10.9,0.0,0.0,0.0,0.0,0.0,65000.0,210000.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,39.0,18.0,3.0,855.0,1
69,9990010000205275,22726.0,181998.0,132176.0,0.0,12600.0,76200.0,28900.0,0.0,1.0,4.0,2.0,0.0,1719.8,5806.8,2988.6,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,57.0,59.0,25.0,33.0,311.0,1
75,9990010000206986,24632.0,38906.0,35179.0,0.0,7600.0,12700.0,1200.0,0.0,2.0,2.0,2.0,0.0,357.6,2263.8,5298.2,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,9.0,18.0,271.0,1
128,9990010000335222,204632.0,316265.0,157903.0,0.0,9200.0,29700.0,6800.0,0.0,1.0,3.0,1.0,0.0,14.3,52.8,27.1,0.0,0.0,0.0,0.0,0.0,0.0,35000.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,7.0,1.0,284.0,1
133,9990010000345234,3218409.0,676798.0,206358.0,0.0,829200.0,159200.0,84100.0,0.0,27.0,9.0,3.0,0.0,286.2,3272.6,3228.8,0.0,0.0,2.0,2.0,0.0,475000.0,140000.0,52000.0,0.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,9.0,11.0,12.0,0.0,973.0,1
135,9990010000350751,65724.0,109904.0,436325.0,0.0,4100.0,4800.0,15500.0,0.0,1.0,1.0,7.0,0.0,30.2,31.9,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0,0.0,279.0,1
139,9990010000355600,493537.0,138541.0,693352.0,0.0,75300.0,8200.0,85000.0,0.0,2.0,1.0,4.0,0.0,57.6,15.2,448.5,0.0,0.0,0.0,3.0,0.0,60000.0,0.0,106000.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,9.0,35.0,4.0,81.0,1
187,9990010000458994,68881.0,49717.0,49027.0,0.0,200.0,200.0,2000.0,0.0,2.0,1.0,1.0,0.0,1376.7,1570.7,2272.7,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,972.0,1
244,9990010000563368,85496.0,136679.0,128313.0,0.0,16500.0,23900.0,12600.0,0.0,1.0,1.0,1.0,0.0,15.0,25.2,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,880.0,1
292,9990010000691400,198478.0,47362.0,55581.0,0.0,4300.0,200.0,200.0,0.0,4.0,1.0,1.0,0.0,50.6,23.9,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,966.0,1


## Random Forest

### not oversampling version without threshold 2/3 or 3/3

In [26]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-02'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-02'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-11'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-02'] == 0))
)

condition_3_3 = ( 
          (df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
          (df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-01'] > 0) & \
          (df_tto_re_pv['TRO_NET 2022-02'] == 0) 
)



df_tto_re_pv['label'] = np.where(condition_3_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2022-02', 
'TRO_DH_SPESIAL 2022-02',
'TRO_NO_ORDER 2022-02',
'SLA 2022-02',
'SLA_LATE 2022-02',
'RATING_BELOW 2022-02',
'RATING_AVG 2022-02',
'PV 2022-02',
'TRO_VOUCHER_USAGE 2022-02',
'TRO_PAYMENT_METHOD 2022-02',
'label'
], 1
)
y = df_tto_re_pv[['label']]


# X['NET-1'] = X.iloc[:,2] - X.iloc[:,1]
# X['NET-2'] = X.iloc[:,1] - X.iloc[:,0]


# X['ORDER-1'] = X.iloc[:,8] - X.iloc[:,7]
# X['ORDER-2'] = X.iloc[:,7] - X.iloc[:,6]


# X['SLA-1'] = X.iloc[:,11] - X.iloc[:,10]
# X['SLA-2'] = X.iloc[:,10] - X.iloc[:,9]


# X['PV-1'] = X.iloc[:,29] - X.iloc[:,28]
# X['PV-2'] = X.iloc[:,28] - X.iloc[:,27]


# skf = StratifiedKFold(n_splits=5)
# for train_index, test_index in skf.split(X, y):
#     print("="*50)
#     X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]


    
#     ## applying standard scaler
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     #clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
#     clf_ridge.fit(X_train, y_train) 
#     pred = clf_ridge.predict(X_test)
#     print(classification_report(y_test, pred))
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ConfusionMatrixDisplay.from_predictions(y_test, pred, 
#                                             cmap='Blues', display_labels= ['not_churn', 'churn'], 
#                                             ax = ax, values_format = 'd')




# applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_70_feb22_3_3.joblib')


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_70_feb22_3_3.joblib')


['./model/mlp_70_feb22_3_3.joblib']

In [ ]:
# a = pd.DataFrame()

# a['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-11']
# a['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# a['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-10']

# a['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# a['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['PV 11-10'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-11']
# a['PV 10-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# a['PV 11-09'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-10']

# a['SLA 11-10'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-11']
# a['SLA 10-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# a['SLA 11-09'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-10']

# a['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-12']/df_tto_re_pv['TRO_NO_ORDER 2021-12']
# a['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [54]:
X_12 = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-11', 
'TRO_DH_SPESIAL 2021-11',
'TRO_NO_ORDER 2021-11',
'SLA 2021-11',
'SLA_LATE 2021-11',
'RATING_BELOW 2021-11',
'RATING_AVG 2021-11',
'PV 2021-11',
'TRO_VOUCHER_USAGE 2021-11',
'TRO_PAYMENT_METHOD 2021-11',
'label'
], 1
)
X_12['SINCE_REGIS'] = X_12['SINCE_REGIS']+30

# X_12 = pd.concat([X_12.iloc[:,0:-16],a],1)
# X_12 = X_12.fillna(0)
# y_12 = df_tto_re_pv['label']

In [55]:
# X_12.head()

In [74]:


scaler_all = load('./model/scaler_70_feb22_3_3.joblib')
clf_all = load('./model/mlp_70_feb22_3_3.joblib')


X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.52).astype(int)


df_mem_12 = pd.concat([df_tto_re_pv[['TRO_MEMBERS']].reset_index(drop=True)
                       , pd.DataFrame(pred, columns=['PRED'])], 1)

In [75]:
df_mem_12[df_mem_12['PRED'] == 1].shape

(9696, 2)

In [76]:
df_trader = pd.read_excel('/home/server/gli-data-science/ferina/Sales Trader Ytd Aug-ExEmployee.xlsx'
                          , dtype='object', names=['member','stat'])

In [77]:
df_mem_12 = df_mem_12[~df_mem_12['TRO_MEMBERS'].isin(df_trader['member'])]

In [78]:
df_mem_12[df_mem_12['PRED'] == 1].shape

(9189, 2)

In [79]:

from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

engine = create_engine(engine_stmt)

df = df_mem_12[df_mem_12['PRED'] == 1].astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn_target', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

In [80]:
## REMOVING NON TRANSACTION MEMBER IN MARCH

# pf = '/home/server/gli-data-science/data/trans_oshop_test/01Mar22_01Apr22.csv'
# df_feb = pd.read_csv(pf, sep='\t', dtype='object')

# df_feb_mem = df_feb['TRO_MEMBERS'].drop_duplicates()

# df_mem_12 = df_mem_12[~df_mem_12['TRO_MEMBERS'].isin(df_feb_mem)]

# df_churn_sel = df_mem_12[df_mem_12['PRED'] == 1].copy()


q = '''
SELECT DISTINCT(tct.TRO_MEMBERS)
FROM TEMP_CHURN_TARGET tct
WHERE 
tct.TRO_MEMBERS NOT IN (
	SELECT rto.TRO_MEMBERS
	FROM RPT_TRANS_OSHOP rto  
	WHERE rto.TRO_DATE BETWEEN '1-mar-22' AND '31-mar-22'
)
AND tct.TRO_MEMBERS NOT IN (
	SELECT 
		gua.GUA_ACCOUNT_CARD_ADJ
	FROM GLI_UNINSTALL_ALFAGIFT gua 
	WHERE 
		gua.GUA_STATUS = 'Y'
		AND trunc(GUA_UNINSTALL_DATE) BETWEEN '01-jul-21' AND '31-mar-22'
)


'''

con = ds_db.connect_alfabi()
df_churn_sel = pd.read_sql_query(q, con)
con.close()

You are connected


In [81]:
df_churn_sel.shape

(4910, 1)

In [82]:
df_churn_sel.head()

,TRO_MEMBERS
0,9990012491849050
1,9990012498724832
2,9990012498996280
3,9990012502431597
4,9990012506226796


In [83]:
df_churn_sel['PERIOD'] = '2022-03-01'

In [84]:
df_churn_sel['PERIOD'] = pd.to_datetime(df_churn_sel['PERIOD'])

In [85]:
df_churn_sel = df_churn_sel[['PERIOD','TRO_MEMBERS']].rename(columns={'TRO_MEMBERS':'MEMBER'})

In [86]:
df_churn_sel.head()

,PERIOD,MEMBER
0,2022-03-01,9990012491849050
1,2022-03-01,9990012498724832
2,2022-03-01,9990012498996280
3,2022-03-01,9990012502431597
4,2022-03-01,9990012506226796


In [87]:
# df_churn_sel_3_3 = df_churn_sel.copy()

In [88]:
df_churn_all = df_churn_sel.copy()

In [92]:
df_churn_all.shape

(4910, 2)

In [ ]:
# df_churn_sel_3_3.tail()

In [ ]:
# df_churn_all = pd.concat([df_churn_sel, df_churn_sel_3_3])
# df_churn_all = df_churn_all.drop_duplicates(subset='MEMBER', keep='last')
# df_churn_all = df_churn_all.iloc[-5000:,:]

In [96]:


# from sqlalchemy import event,create_engine,types



# driver = 'cx_oracle'
# server = '10.234.152.61' 
# database = 'alfabi' 
# username = 'mkt' 
# password = 'Mkt234'
# engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )
    
# engine = create_engine(engine_stmt)

# q = '''

# DELETE FROM ALFAGIFT_CHURN
# WHERE PERIOD = '1-mar-22'
# '''

# con = engine.connect()
# con.execute(q)

# df = df_churn_all.copy()
# dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
# df.to_sql('ALFAGIFT_CHURN', engine, index=False, if_exists="append", dtype=dtyp)



# engine.dispose()

In [93]:
df_churn_view = pd.merge(df_churn_sel,df_tto_re_pv,left_on='MEMBER',right_on='TRO_MEMBERS', how='left')

In [94]:
df_churn_view.head(100)

,PERIOD,MEMBER,TRO_MEMBERS,TRO_NET 2021-11,TRO_NET 2021-12,TRO_NET 2022-01,TRO_NET 2022-02,TRO_DH_SPESIAL 2021-11,TRO_DH_SPESIAL 2021-12,TRO_DH_SPESIAL 2022-01,TRO_DH_SPESIAL 2022-02,TRO_NO_ORDER 2021-11,TRO_NO_ORDER 2021-12,TRO_NO_ORDER 2022-01,TRO_NO_ORDER 2022-02,SLA 2021-11,SLA 2021-12,SLA 2022-01,SLA 2022-02,SLA_LATE 2021-11,SLA_LATE 2021-12,SLA_LATE 2022-01,SLA_LATE 2022-02,TRO_VOUCHER_USAGE 2021-11,TRO_VOUCHER_USAGE 2021-12,TRO_VOUCHER_USAGE 2022-01,TRO_VOUCHER_USAGE 2022-02,TRO_PAYMENT_METHOD 2021-11,TRO_PAYMENT_METHOD 2021-12,TRO_PAYMENT_METHOD 2022-01,TRO_PAYMENT_METHOD 2022-02,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_BELOW 2022-01,RATING_BELOW 2022-02,RATING_AVG 2021-11,RATING_AVG 2021-12,RATING_AVG 2022-01,RATING_AVG 2022-02,PV 2021-11,PV 2021-12,PV 2022-01,PV 2022-02,SINCE_REGIS,label
0,2022-03-01,9990012491849050,9990012491849050,0.0,17272.0,34545.0,34545.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,3.8,41.1,88.1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,830.0,0
1,2022-03-01,9990012498724832,9990012498724832,0.0,119486.0,333934.0,12817.0,0.0,14800.0,8700.0,200.0,0.0,2.0,12.0,1.0,0.0,52.6,29053.1,0.8,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,7.0,7.0,904.0,0
2,2022-03-01,9990012498996280,9990012498996280,18271.0,25181.0,50181.0,13700.0,600.0,200.0,200.0,200.0,1.0,1.0,1.0,1.0,4453.6,1740.6,135.8,3083.8,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,903.0,0
3,2022-03-01,9990012502431597,9990012502431597,0.0,34090.0,20817.0,13000.0,0.0,0.0,400.0,0.0,0.0,1.0,1.0,1.0,0.0,1711.9,39.6,12.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,881.0,0
4,2022-03-01,9990012506226796,9990012506226796,0.0,66726.0,69272.0,37635.0,0.0,400.0,400.0,3400.0,0.0,1.0,1.0,1.0,0.0,1.8,3.6,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,839.0,0
5,2022-03-01,9990012510360726,9990012510360726,0.0,223173.0,235445.0,105177.0,0.0,34700.0,11300.0,9800.0,0.0,1.0,2.0,1.0,0.0,2163.3,289.2,68.1,0.0,1.0,1.0,1.0,0.0,3800000.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,4.0,40.0,5.0,874.0,0
6,2022-03-01,9990012515738800,9990012515738800,115451.0,199113.0,37271.0,13681.0,4200.0,4500.0,200.0,200.0,2.0,1.0,1.0,1.0,16.4,1848.7,3574.3,1.5,0.0,1.0,1.0,0.0,0.0,3400000.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,894.0,0
7,2022-03-01,9990012520582700,9990012520582700,210129.0,28300.0,85909.0,68136.0,22600.0,0.0,0.0,7300.0,3.0,1.0,5.0,1.0,97.5,17.5,100.0,80.7,1.0,0.0,0.0,1.0,0.0,0.0,0.0,18000.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,879.0,0
8,2022-03-01,9990012521854270,9990012521854270,0.0,36381.0,55723.0,25062.0,0.0,200.0,9000.0,200.0,0.0,1.0,1.0,1.0,0.0,21.3,3784.4,1757.1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,873.0,0
9,2022-03-01,9990012522739900,9990012522739900,66726.0,222860.0,118545.0,59181.0,400.0,23200.0,600.0,400.0,1.0,2.0,1.0,1.0,10.9,143.9,47.3,6.5,0.0,1.0,0.0,0.0,0.0,1000000.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,17.0,18.0,6.0,868.0,0
